# Construct the UNet Model with brevitas

In [12]:
import torch.nn.functional as F
import brevitas.nn as bnn
from brevitas.nn import QuantConv2d, QuantLinear, QuantReLU, QuantIdentity
from brevitas.quant import Int8WeightPerTensorFixedPoint, Int8ActPerTensorFixedPoint
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from torch.utils.data import DataLoader
import torch
import onnx
from finn.util.test import get_test_model_trained
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
import os

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
root_path = './mybuild/emerald_video_denoise_acc'
data_path = root_path + '/data'
MODEL_PATH = root_path + '/model/quant_vgg13_cifar10.pth'
BATCH_SIZE = 128
EPOCHS = 100
LR = 0.001

# ceate directories if they do not exist
os.makedirs(data_path, exist_ok=True)
os.makedirs(root_path + '/model', exist_ok=True)


In [15]:
class Denoise(nn.Module):
    def __init__(self):
        super(Denoise, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            bnn.QuantConv2d(1, 16, kernel_size=3, stride=2, padding=1,weight_bit_width=4),  
            bnn.QuantReLU(bit_width=4),
            bnn.QuantConv2d(16, 8, kernel_size=3, stride=2, padding=1,weight_bit_width=4),  
            bnn.QuantReLU(bit_width=4)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            bnn.QuantConvTranspose2d(8, 8, kernel_size=3, stride=2, padding=1, weight_bit_width=4),#, output_padding = 1
            bnn.QuantReLU(),
            bnn.QuantConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, weight_bit_width=4), #, output_padding = 1
            bnn.QuantReLU(),
            bnn.QuantConv2d(16, 1, kernel_size=3, padding=1),  
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
model = Denoise()
model = model.float()
print(model)

Denoise(
  (encoder): Sequential(
    (0): QuantConv2d(
      1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
      (input_quant): ActQuantProxyFromInjector(
        (_zero_hw_sentinel): StatelessBuffer()
      )
      (output_quant): ActQuantProxyFromInjector(
        (_zero_hw_sentinel): StatelessBuffer()
      )
      (weight_quant): WeightQuantProxyFromInjector(
        (_zero_hw_sentinel): StatelessBuffer()
        (tensor_quant): RescalingIntQuant(
          (int_quant): IntQuant(
            (float_to_int_impl): RoundSte()
            (tensor_clamp_impl): TensorClampSte()
            (delay_wrapper): DelayWrapper(
              (delay_impl): _NoDelay()
            )
            (input_view_impl): Identity()
          )
          (scaling_impl): StatsFromParameterScaling(
            (parameter_list_stats): _ParameterListStats(
              (first_tracked_param): _ViewParameterWrapper(
                (view_shape_impl): OverTensorView()
              )
              (st